In [ ]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
# store url in a variable
url = "https://en.wikipedia.org/wiki/Berlin"

In [ ]:
# download html with a get request
headers = {'Accept-Language': 'en-US,en;q=0.8'}

response = requests.get(url, headers = headers)

response.status_code

200

In [ ]:
# parse html (create the 'soup')
wiki_soup_be = BeautifulSoup(response.content, "html.parser")

wiki_soup_be

# Info. we want to extract:

1. City name
2. Country name
3. Latitude
4. Longitude
5. Population

In [ ]:
# extracting city name

city_name_0 = wiki_soup_be.select("div.fn.org")[0].get_text()

In [ ]:
city_name_0

'Berlin'

In [ ]:
# extracting city name
city_name = wiki_soup_be.select("span.mw-page-title-main")[0].get_text()
city_name

'Berlin'

In [ ]:
# extracting country name
country_name = wiki_soup_be.select("td.infobox-data a")[0].get_text()
country_name

'Germany'

In [ ]:
# extracting latitude
latitude = wiki_soup_be.select("span.latitude")[0].get_text()
latitude

'52°31′12″N'

In [ ]:
# extracting longitude
longitude = wiki_soup_be.select("span.longitude")[0].get_text()
longitude

'13°24′18″E'

In [ ]:
# extracting population
population_0 = wiki_soup_be.select("td.infobox-data")[10].get_text()
population_0

'3,850,809'

In [ ]:
import re

In [ ]:
population_1 = wiki_soup_be.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
population_1

<ipython-input-13-51a0ff374a11>:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  population_1 = wiki_soup_be.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))


'3,850,809'

In [ ]:
list_of_cities = ["Frankfurt", "Berlin", "Cologne", "Munich", "Hamburg"]

In [ ]:
def webscrape_wiki(list_of_cities):

  list_for_df = []

  for city in list_of_cities:

    url = "https://en.wikipedia.org/wiki/" + city

    headers = {'Accept-Language': 'en-US,en;q=0.8'}
    response = requests.get(url, headers = headers)

    wiki_soup = BeautifulSoup(response.content, "html.parser")

    response_dict = {}

    response_dict["city"] = wiki_soup.select("span.mw-page-title-main")[0].getText()
    response_dict["country"] = wiki_soup.select("table.infobox td.infobox-data")[0].getText()
    response_dict["latitude"] = wiki_soup.select("span.latitude")[0].getText()
    response_dict["longitude"] = wiki_soup.select("span.longitude")[0].getText()

    if wiki_soup.select_one('th.infobox-header:-soup-contains("Population")'):
      response_dict["population"] = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

    list_for_df.append(response_dict)

  cities_df = pd.DataFrame(list_for_df)



  return cities_df

In [ ]:
webscrape_wiki(list_of_cities)

<ipython-input-17-31dc67c0ca27>:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  response_dict["population"] = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))


,city,country,latitude,longitude,population
0,Frankfurt,Germany,50°06′38″N,08°40′56″E,"773,068"
1,Berlin,Germany,52°31′12″N,13°24′18″E,"3,850,809"
2,Cologne,Germany,50°56′11″N,6°57′10″E,"1,073,096"
3,Munich,Germany,48°08′15″N,11°34′30″E,"1,512,491"
4,Hamburg,Germany,53°33′N,10°00′E,"1,906,411"


In [ ]:
# lat = '''51°36'9.18"N'''
# deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
# (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

In [ ]:
scooter_cities_df = pd.DataFrame(
    {"city_name": [],
     "country": [],
     "latitude": [],
     "longitude": [],
     "population": []
    }
)

for city in list_of_cities:

    url = f"https://en.wikipedia.org/wiki/{city}"

    headers = {'Accept-Language': 'en-US,en;q=0.8'}
    response = requests.get(url, headers = headers)

    if response.status_code != 200: break

    wiki_soup = BeautifulSoup(response.content, "html.parser")

    city_name = wiki_soup.select("span.mw-page-title-main")[0].getText()
    country_name = wiki_soup.select("table.infobox td.infobox-data")[0].getText()
    latitude = wiki_soup.select("span.latitude")[0].getText()
    longitude = wiki_soup.select("span.longitude")[0].getText()

    if wiki_soup.select_one('th.infobox-header:-soup-contains("Population")'):
        population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

    city_df = pd.DataFrame(
        {"city_name": [city_name],
         "country": [country_name],
         "latitude": [latitude],
         "longitude": [longitude],
         "population": [population]
        }
    )

    scooter_cities_df = pd.concat([scooter_cities_df, city_df], ignore_index = True)

    # fixing latitude
    scooter_cities_df["latitude"] = scooter_cities_df["latitude"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    # fixing longitude
    scooter_cities_df["longitude"] = scooter_cities_df["longitude"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    # fixing population
    scooter_cities_df["population"] = scooter_cities_df["population"].str.replace(',', '', regex=False)


scooter_cities_df

<ipython-input-16-e1b53b30c850>:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
<ipython-input-16-e1b53b30c850>:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
<ipython-input-16-e1b53b30c850>:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
<ipython-input-16-e1b53b30c850>:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  population = wiki_soup.se

,city_name,country,latitude,longitude,population
0,Frankfurt,Germany,50.0638,08.4056,773068
1,Berlin,Germany,52.3112,13.2418,3850809
2,Cologne,Germany,50.5611,6.5710,1073096
3,Munich,Germany,48.0815,11.3430,1512491
4,Hamburg,Germany,53.33N,10.00E,1906411
